# 2489-2122 Data Curation  T1 Final Exam 

The final exam will contain 1 question with subquestions for 70% of the total points.

## A Million Dollar Question: Squid Game or Alice in Borderland?


“What TV sereis should I binge-watch this evening?” This perhaps is a question you would ask yourself very often. As for me — yes, and more than once. As such, from Netflix to Hulu, the need to build robust movie recommendation systems is extremely important given the huge demand for personalized content of modern consumers. **Netflix is forecasting it will add 3.5 million paying subscribers thanks to the surprise hit Squid Games**

We are going to examine a MovieLens dataset which provides non-commercial, personalized movie recommendations. 

This dataset describes user ratings from MovieLens. It contains ratings and tag applications across movies created by  users. Users were selected at random for inclusion. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files movies_fe.xlsx, ratings_fe.csv. More details about the contents and use of all these files follows.

**Ratings Data File Structure (ratings_fe.csv)**
All ratings are contained in the file ratings_fe.csv. Each line of this file after the header row represents one rating of one movie by one user, and has the following format:
`userId,movieId,rating,timestamp`

The lines within this file are ordered first by userId, then, within user, by movieId.

Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

**Movies Data File Structure (movies_fe.xlsx)**
Movie information is contained in the file movies_fe.xlsx. Each line of this file after the header row represents one movie, and has the following format:
`movieId,title,year,genres`

Answer the following questions using the provided dataset. You can write down intermediate results towards the final answers

In [2]:
import pandas as pd
import numpy as np

### Question 1 (10 points)

However, errors and inconsistencies may exist in these files shown as below:

The ratings in the `rating_fe.csv` should be made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars). So if the ratings that are larger than 5 or smaller than 0.5, you need to round it to the value of 5 and 1, respectively. For example, if a movie is rated 8, then it might be wronly rated and you need to change the value to 5. Similarly, if a movie is rated negatively, e.g., -1, then it should be changed to 1, if any.

The movie information in the `movies_fe.xlsx` contains the movies with the missing information about **year**. You should also remove them.

You should also inspect the data to make sure the correct starting row of the data. 

In [30]:
ratings = pd.read_csv('ratings_fe.csv', skiprows=10, index_col=0, encoding = "ISO-8859-1")

ratings["rating"] = ratings["rating"].replace([list(ratings[ratings["rating"]>5.0]["rating"].value_counts().index)],5.0).replace([list(ratings[ratings["rating"]<0.5]["rating"].value_counts().index)],1.0)

In [157]:
movies = pd.read_excel('movies_fe.xlsx', skiprows=15)
movies.dropna(subset=["year"])

,movieId,title,year,genres
0,1,Toy Story,1995.0,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,1995.0,Adventure|Children|Fantasy
2,3,Grumpier Old Men,1995.0,Comedy|Romance
3,4,Waiting to Exhale,1995.0,Comedy|Drama|Romance
4,5,Father of the Bride Part II,1995.0,Comedy
...,...,...,...,...
8564,123109,P.U.N.K.S,1999.0,Children|Comedy|Sci-Fi
8565,124857,Deception,2013.0,Action
8566,125916,Fifty Shades of Grey,2015.0,Drama
8567,126407,Face of Terror,2005.0,Action|Drama|Thriller


### Question 2 (10 points)

Show the top 5 Action movies with the highest median ratings:

In [158]:
pd.merge(movies, ratings.groupby("movieId").median().drop(columns="timestamp", axis=1).reset_index(), on="movieId").sort_values("rating", ascending=False)[:5]


,movieId,title,year,genres,userId,rating
591,678,Some Folks Call It a Sling Blade,1993.0,Drama|Thriller,298.5,5.0
563,632,Land and Freedom (Tierra y libertad),1995.0,Drama|War,373.0,5.0
5473,8491,White Heat,1949.0,Crime|Drama|Film-Noir,41.5,5.0
3310,4227,"Brothers, The",2001.0,Comedy|Drama,111.0,5.0
5851,26712,35 Up,1991.0,Documentary,435.0,5.0


### Question 3 (15 points)

Among all movies that user with Id 500 has rated, show the his/her top 5 favorite movies in each of the following three genres **Adventure**, **Comedy**, **Drama** (i.e., the movie he/she rated 5) more recently as three columns: `movieId, title, genre`. If you see the movies with overlapping genres, it is ok to include them several times.

In [186]:
fullDF=pd.merge(movies, ratings, on="movieId")
id500=fullDF[fullDF["userId"]==500]
id500av = id500[id500["genres"].str.contains("Adventure")]
id500dr = id500[id500["genres"].str.contains("Drama")]
id500co = id500[id500["genres"].str.contains("Comedy")]
id500av = id500av.sort_values(["rating","timestamp"])[["movieId", "title", "genres"]][:5]
id500dr = id500dr.sort_values(["rating","timestamp"])[["movieId", "title", "genres"]][:5]
id500co = id500co.sort_values(["rating","timestamp"])[["movieId", "title", "genres"]][:5]
print("For Drama: ")
print(id500dr["title"])
print("\nFor Adventure: ")
print(id500av["title"])
print("\nFor Comedy: ")
print(id500co["title"])

For Drama: 
82803             Core, The
21126    Dances with Wolves
76548               Glitter
42058                Shiloh
82815     What a Girl Wants
Name: title, dtype: object

For Adventure: 
79950             Scooby-Doo
78930      Time Machine, The
21126     Dances with Wolves
29032    Alice in Wonderland
50334      Song of the South
Name: title, dtype: object

For Comedy: 
79950                       Scooby-Doo
37710               This Is Spinal Tap
39989                         Grease 2
9071     Kid in King Arthur's Court, A
57158               Night of the Comet
Name: title, dtype: object


### Question 4 (15 points)

Show the pivot table of mean and standard deviation for the ratings of movies across the row of released decades (for example, year 1995 belongs to 1990s decade), and the column of quartile of the timestamp values (in terms of 4 groups).

In [160]:
fullDF["decade"] = pd.qcut(fullDF["year"],3,labels=["1990s","2000s","2010s"])
fullDF["timestampq"] = pd.qcut(fullDF["timestamp"],4,labels=["1st","2nd","3rd","4th"])

In [175]:
m=fullDF.groupby("decade").mean().reset_index()[["decade","rating"]]
std=fullDF.groupby("decade").std().reset_index()[["decade","rating"]]
stdm = pd.merge(m, std, on="decade").rename(columns={"rating_x":"mean","rating_y":"std"})
o=pd.merge(fullDF, stdm, on="decade").groupby(["decade","timestampq"]).mean()[["mean","std"]].reset_index()
pd.pivot_table(o, columns=["timestampq","decade"])

timestampq       1st                           2nd                      \
decade         1990s     2000s     2010s     1990s     2000s     2010s   
mean        3.605573  3.413230  3.445491  3.605573  3.413230  3.445491   
std         1.055886  1.077834  1.059395  1.055886  1.077834  1.059395   

timestampq       3rd                           4th                      
decade         1990s     2000s     2010s     1990s     2000s     2010s  
mean        3.605573  3.413230  3.445491  3.605573  3.413230  3.445491  
std         1.055886  1.077834  1.059395  1.055886  1.077834  1.059395

### Question 5 (20 points)

Now you need to implement a **recommender system using collaborative filtering method**. This works simply as to recommend movies that "people who like this movie also like these movies". For example, people who like to watch Star Wars are very likely to watch Star Treks. 

In order to do so, you need to find all users who like one movie (i.e., post a rating of 5), and identify the movies these users also like, ranked by the number of likes. 

Show the recommended movie list with top 10 movies that users who like the *Titanic* may also like.

In [181]:
users = fullDF[(fullDF["rating"]==5.0) & (fullDF["title"]=="Titanic")]["userId"]
users
liked_movies = fullDF[(fullDF["rating"]==5.0) & (fullDF["title"]!="Titanic") & fullDF["userId"].isin(users)]
this = liked_movies.groupby("movieId")["userId"].count().reset_index().rename(columns={"userId":"times_rated"}).sort_values(by='times_rated', 
                                                                                                               ascending=False)

pd.merge(movies, this, on="movieId").sort_values("times_rated", ascending=False)[:10]

,movieId,title,year,genres,times_rated
90,593,"Silence of the Lambs, The",1991.0,Crime|Horror|Thriller,8
344,2571,"Matrix, The",1999.0,Action|Sci-Fi|Thriller,8
51,356,Forrest Gump,1994.0,Comedy|Drama|Romance|War,8
76,527,Schindler's List,1993.0,Drama|War,8
45,318,"Shawshank Redemption, The",1994.0,Crime|Drama,8
88,589,Terminator 2: Judgment Day,1991.0,Action|Sci-Fi,7
22,110,Braveheart,1995.0,Action|Drama|War,7
280,2028,Saving Private Ryan,1998.0,Action|Drama|War,7
256,1784,As Good as It Gets,1997.0,Comedy|Drama|Romance,7
382,2959,Fight Club,1999.0,Action|Crime|Drama|Thriller,6


Congratulations! You just build the first [recommender system that worth 1 million dollars](https://www.netflixprize.com/) :D

![netflix_prize](https://cdn.vox-cdn.com/thumbor/Kp9TEknNzIQV-ZijAm74cfHx_D0=/0x124:1100x700/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/15788062/netflix-prize1.0.1537040369.jpg)